<a href="https://colab.research.google.com/github/Gauravsaxena816/LanguageDetection/blob/main/minorpbl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np


In [3]:
import pandas as pd

file_path = "/content/minorbaka.csv"
df = pd.read_csv(file_path)
df.head()

,L,LIL,LIR,mob_IL,mob_IR,mun_0,mup_0,phi_left,phi_right,IL_dop,...,Jsc0.56,Jsc1.00,FF0.10,FF0.18,FF0.32,FF0.56,FF1.00,label,Rseries,Rshunt
0,5.108427e-07,9.676050e-08,7.962649e-08,1.814331e-06,4.974563e-07,0.004376,0.000189,0.199219,0.199061,0.000000e+00,...,-117.1847,-208.3871,0.8556,0.8562,0.8467,0.8255,0.7886,0.0,0.000339,4.111016
1,6.812739e-07,3.104342e-08,6.971102e-08,1.041681e-06,1.045686e-06,0.000077,0.007106,0.118199,0.173381,-2.784254e+21,...,-120.8212,-214.8071,0.8413,0.8104,0.7546,0.6571,0.4971,0.0,0.002586,90.277880
2,5.489128e-07,2.870512e-08,2.911453e-08,6.059205e-06,2.757981e-07,0.004711,0.000310,0.105066,0.063592,0.000000e+00,...,-120.1723,-213.6999,0.9038,0.9040,0.9037,0.9028,0.9007,0.0,0.000000,1000.000000
3,4.402036e-07,2.575070e-08,3.330055e-08,6.212196e-06,1.335840e-08,0.038179,0.000296,0.073770,0.130407,0.000000e+00,...,-118.2851,-210.3439,0.8622,0.8466,0.8240,0.7933,0.7529,0.0,0.000082,36.328948
4,6.087204e-07,7.754248e-08,8.818968e-08,6.051171e-07,4.080898e-08,0.029594,0.035290,0.081791,0.089885,0.000000e+00,...,-121.9797,-216.9140,0.8565,0.8280,0.7786,0.6976,0.5730,0.0,0.001169,23.957157


In [4]:
#new column Jsc, and Voc that median of Jsc0.10,Jsc0.18,Jsc0.32,Jsc0.56,Jsc1.0, and for voc median of voc0.10,voc0.18,voc0.32,voc0.56,voc1.0

jsc_cols = ['Jsc0.10', 'Jsc0.18', 'Jsc0.32', 'Jsc0.56', 'Jsc1.00']
voc_cols = ['Voc0.10', 'Voc0.18', 'Voc0.32', 'Voc0.56', 'Voc1.00']

df['Jsc'] = df[jsc_cols].median(axis=1)
df['Voc'] = df[voc_cols].median(axis=1)
df.head()

,L,LIL,LIR,mob_IL,mob_IR,mun_0,mup_0,phi_left,phi_right,IL_dop,...,FF0.10,FF0.18,FF0.32,FF0.56,FF1.00,label,Rseries,Rshunt,Jsc,Voc
0,5.108427e-07,9.676050e-08,7.962649e-08,1.814331e-06,4.974563e-07,0.004376,0.000189,0.199219,0.199061,0.000000e+00,...,0.8556,0.8562,0.8467,0.8255,0.7886,0.0,0.000339,4.111016,-65.8978,1.2817
1,6.812739e-07,3.104342e-08,6.971102e-08,1.041681e-06,1.045686e-06,0.000077,0.007106,0.118199,0.173381,-2.784254e+21,...,0.8413,0.8104,0.7546,0.6571,0.4971,0.0,0.002586,90.277880,-67.9479,1.2485
2,5.489128e-07,2.870512e-08,2.911453e-08,6.059205e-06,2.757981e-07,0.004711,0.000310,0.105066,0.063592,0.000000e+00,...,0.9038,0.9040,0.9037,0.9028,0.9007,0.0,0.000000,1000.000000,-67.5778,1.3235
3,4.402036e-07,2.575070e-08,3.330055e-08,6.212196e-06,1.335840e-08,0.038179,0.000296,0.073770,0.130407,0.000000e+00,...,0.8622,0.8466,0.8240,0.7933,0.7529,0.0,0.000082,36.328948,-66.5166,1.2865
4,6.087204e-07,7.754248e-08,8.818968e-08,6.051171e-07,4.080898e-08,0.029594,0.035290,0.081791,0.089885,0.000000e+00,...,0.8565,0.8280,0.7786,0.6976,0.5730,0.0,0.001169,23.957157,-68.5943,1.3382


In [5]:
columns_to_remove = ['L', 'LIL', 'LIR', 'mob_IL','mob_IR', 'IL_dop','IR_dop', 'Lang_pre', 'St_L', 'St_R', 'n', 'Cn','Cp', 'Gmax', 'Jsc0.10', 'Jsc0.18', 'Jsc0.32', 'Jsc0.56', 'Jsc1.00', 'voc0.10', 'voc0.18', 'voc0.32', 'voc0.56', 'voc1.00']
df = df.drop(columns=columns_to_remove, errors='ignore')
df.head()


,mun_0,mup_0,phi_left,phi_right,Etrap,GB_tr,num_GBs,R2w,aplhaw,alphaR2w,...,FF0.10,FF0.18,FF0.32,FF0.56,FF1.00,label,Rseries,Rshunt,Jsc,Voc
0,0.004376,0.000189,0.199219,0.199061,0.637334,0.0,2.0,0.999987,1.000000,1.0,...,0.8556,0.8562,0.8467,0.8255,0.7886,0.0,0.000339,4.111016,-65.8978,1.2817
1,0.000077,0.007106,0.118199,0.173381,0.694056,0.0,2.0,0.999999,0.999756,1.0,...,0.8413,0.8104,0.7546,0.6571,0.4971,0.0,0.002586,90.277880,-67.9479,1.2485
2,0.004711,0.000310,0.105066,0.063592,0.584025,0.0,2.0,0.999999,1.000000,1.0,...,0.9038,0.9040,0.9037,0.9028,0.9007,0.0,0.000000,1000.000000,-67.5778,1.3235
3,0.038179,0.000296,0.073770,0.130407,0.443934,0.0,2.0,0.999997,1.000000,1.0,...,0.8622,0.8466,0.8240,0.7933,0.7529,0.0,0.000082,36.328948,-66.5166,1.2865
4,0.029594,0.035290,0.081791,0.089885,0.539406,0.0,2.0,0.999999,1.000000,1.0,...,0.8565,0.8280,0.7786,0.6976,0.5730,0.0,0.001169,23.957157,-68.5943,1.3382


In [7]:
#Analyzing data
df.describe()

,mun_0,mup_0,phi_left,phi_right,Etrap,GB_tr,num_GBs,R2w,aplhaw,alphaR2w,...,FF0.10,FF0.18,FF0.32,FF0.56,FF1.00,label,Rseries,Rshunt,Jsc,Voc
count,81662.000000,81662.000000,81661.000000,81661.000000,81661.000000,8.166100e+04,81661.000000,81661.000000,81661.000000,8.166100e+04,...,81661.000000,81661.000000,81661.000000,81661.000000,81661.000000,81661.0,81661.000000,81660.000000,81661.000000,81661.000000
mean,0.007090,0.007125,0.099417,0.099767,0.599828,-2.109235e+11,1.983358,0.999822,0.999998,1.000000e+00,...,0.831434,0.834756,0.823368,0.797110,0.753970,0.0,0.000391,81.776672,-67.204146,1.303269
std,0.011296,0.011268,0.057562,0.057746,0.115244,1.088828e+13,0.719777,0.000369,0.000022,2.595285e-09,...,0.064738,0.059796,0.075086,0.104395,0.147746,0.0,0.000770,242.979407,1.015528,0.024705
min,0.000050,0.000050,0.000003,0.000007,0.400006,-9.539145e+14,0.000000,0.998001,0.997693,9.999995e-01,...,0.518200,0.587400,0.526400,0.447200,0.350000,0.0,0.000000,0.245241,-69.303300,1.016900
25%,0.000265,0.000273,0.049667,0.049840,0.500464,0.000000e+00,1.000000,0.999859,0.999999,1.000000e+00,...,0.794500,0.797900,0.782700,0.737600,0.666700,0.0,0.000005,1.776895,-67.988000,1.284200
50%,0.001442,0.001511,0.099020,0.099293,0.599363,0.000000e+00,2.000000,0.999988,1.000000,1.000000e+00,...,0.851800,0.850900,0.846900,0.835700,0.807600,0.0,0.000039,7.760080,-67.338500,1.303100
75%,0.008511,0.008727,0.149020,0.149799,0.699437,0.000000e+00,2.000000,0.999998,1.000000,1.000000e+00,...,0.884000,0.884300,0.883400,0.880700,0.875700,0.0,0.000341,33.574224,-66.528600,1.323300
max,0.049986,0.049998,0.199999,0.199999,0.799999,0.000000e+00,3.000000,1.000000,1.000256,1.000000e+00,...,0.906200,0.907000,0.907800,0.908500,0.909100,0.0,0.004468,1000.000000,-61.422400,1.360200


In [9]:
# Assuming 'df' is your preprocessed DataFrame
df.to_csv('minorbaka2.csv', index=False)

In [4]:
import pandas as pd

file_path = "/content/minorbaka2.csv"
df = pd.read_csv(file_path)
df.head()

,mun_0,mup_0,phi_left,phi_right,Etrap,GB_tr,num_GBs,R2w,aplhaw,alphaR2w,...,FF0.10,FF0.18,FF0.32,FF0.56,FF1.00,label,Rseries,Rshunt,Jsc,Voc
0,-0.242156,-0.616386,1.734317,1.720126,0.325200,0.017967,0.020734,0.447364,0.097619,0.016528,...,0.372612,0.357603,0.309397,0.270523,0.233001,0.0,-0.067015,-0.314575,1.336861,-0.918013
1,-0.622296,-0.002881,0.326529,1.275277,0.817518,0.017967,0.020734,0.478055,-12.318245,-1.787862,...,0.150866,-0.411640,-0.921445,-1.346928,-1.744319,0.0,2.858036,0.047568,-0.732642,-2.305948
2,-0.212510,-0.605676,0.098329,-0.626600,-0.137485,0.017967,0.020734,0.478181,0.122384,0.016527,...,1.120037,1.160437,1.071156,1.012974,0.993404,0.0,-0.508762,3.870961,-0.359039,0.829447
3,2.747315,-0.606911,-0.445454,0.530829,-1.353394,0.017967,0.020734,0.473594,0.107626,0.016532,...,0.474957,0.196364,0.006030,-0.038752,-0.009162,0.0,-0.401878,-0.179169,0.712205,-0.717348
4,1.988060,2.496918,-0.306086,-0.171141,-0.524752,0.017967,0.020734,0.478202,0.091957,0.016509,...,0.386568,-0.116036,-0.600704,-0.957933,-1.229470,0.0,1.013316,-0.231165,-1.385160,1.443985


In [5]:
import numpy as np
# Handle missing values (if any)
df.fillna(df.median(), inplace=True) # Fill with median for numerical columns

# Identify and handle outliers (if needed)
# Example using IQR method for 'Jsc' and 'Voc'
Q1_jsc = df['Jsc'].quantile(0.25)
Q3_jsc = df['Jsc'].quantile(0.75)
IQR_jsc = Q3_jsc - Q1_jsc
lower_bound_jsc = Q1_jsc - 1.5 * IQR_jsc
upper_bound_jsc = Q3_jsc + 1.5 * IQR_jsc

Q1_voc = df['Voc'].quantile(0.25)
Q3_voc = df['Voc'].quantile(0.75)
IQR_voc = Q3_voc - Q1_voc
lower_bound_voc = Q1_voc - 1.5 * IQR_voc
upper_bound_voc = Q3_voc + 1.5 * IQR_voc

# Option 1: Remove outliers
df = df[~((df['Jsc'] < lower_bound_jsc) | (df['Jsc'] > upper_bound_jsc))]
df = df[~((df['Voc'] < lower_bound_voc) | (df['Voc'] > upper_bound_voc))]

# Option 2: Cap outliers
#df['Jsc'] = np.clip(df['Jsc'], lower_bound_jsc, upper_bound_jsc)
#df['Voc'] = np.clip(df['Voc'], lower_bound_voc, upper_bound_voc)


# Feature scaling (if needed)
# Example using StandardScaler for numerical features
numerical_cols = df.select_dtypes(include=np.number).columns
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
df.head()


,mun_0,mup_0,phi_left,phi_right,Etrap,GB_tr,num_GBs,R2w,aplhaw,alphaR2w,...,FF0.10,FF0.18,FF0.32,FF0.56,FF1.00,label,Rseries,Rshunt,Jsc,Voc
0,-0.242194,-0.616432,1.734354,1.720215,0.325050,0.017973,0.020697,0.447424,0.099052,0.016170,...,0.372605,0.357568,0.309343,0.270463,0.232946,0.0,-0.067029,-0.314344,1.341031,-0.918816
1,-0.622373,-0.003002,0.326573,1.275348,0.817344,0.017973,0.020697,0.478115,-12.645342,-1.795055,...,0.150838,-0.411784,-0.921656,-1.347129,-1.744484,0.0,2.858196,0.048159,-0.732532,-2.307562
2,-0.212546,-0.605724,0.098373,-0.626610,-0.137613,0.017973,0.020697,0.478242,0.124472,0.016169,...,1.120101,1.160515,1.071200,1.012981,0.993391,0.0,-0.508802,3.875340,-0.358196,0.829666
3,2.747574,-0.606958,-0.445407,0.530868,-1.353463,0.017973,0.020697,0.473655,0.109323,0.016174,...,0.474959,0.196306,0.005937,-0.038839,-0.009230,0.0,-0.401912,-0.178803,0.715149,-0.718033
4,1.988243,2.496497,-0.306039,-0.171132,-0.524861,0.017973,0.020697,0.478262,0.093241,0.016151,...,0.386563,-0.116138,-0.600874,-0.958100,-1.229606,0.0,1.013366,-0.230851,-1.386330,1.444562


In [6]:
# Split data into features (X) and target variables (y)
X = df.drop(['Jsc'], axis=1)
y = df[['Jsc']]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#XGBoost Model
xgb_model = MultiOutputRegressor(XGBRegressor(random_state=42))
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

# Evaluate XGBoost model
xgb_mse = mean_squared_error(y_test, xgb_pred)
xgb_r2 = r2_score(y_test, xgb_pred)

print("XGBoost Results:")
print(f"Mean Squared Error: {xgb_mse}")
print(f"R-squared: {xgb_r2}")




XGBoost Results:
Mean Squared Error: 0.9432346224784851
R-squared: 0.04921591281890869


In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

#Decision Tree Model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_test)

# Evaluate Decision Tree model
dt_mse = mean_squared_error(y_test, dt_pred)
dt_r2 = r2_score(y_test, dt_pred)

print("\nDecision Tree Results:")
print(f"Mean Squared Error: {dt_mse}")
print(f"R-squared: {dt_r2}")



Decision Tree Results:
Mean Squared Error: 1.9080542672543195
R-squared: -0.9233260379425403


In [9]:
#Random Forest Model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# Evaluate Random Forest Model
rf_mse = mean_squared_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

print("\nRandom Forest Results:")
print(f"Mean Squared Error: {rf_mse}")
print(f"R-squared: {rf_r2}")


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Random Forest Results:
Mean Squared Error: 0.9586205651393521
R-squared: 0.03370678440249952


In [10]:
#SVM Model
svm_model = SVR()
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)

# Evaluate SVM Model
svm_mse = mean_squared_error(y_test, svm_pred)
svm_r2 = r2_score(y_test, svm_pred)

print("\nSVM Results:")
print(f"Mean Squared Error: {svm_mse}")
print(f"R-squared: {svm_r2}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



SVM Results:
Mean Squared Error: 0.9825987844907333
R-squared: 0.009536647099029238


In [13]:
!pip install plotly

In [14]:
import plotly.graph_objects as go

In [21]:
fig = go.Figure(data=[go.Bar(
    x=models,
    y=scores,
    text=scores, #add the score values to the plot
    textposition='auto',
    marker_color=['purple', 'red', 'orange', 'blue']
)])

fig.update_layout(
    title="Comparison of R-squared Scores for Different Regression Models (Joc)",
    xaxis_title="Models",
    yaxis_title="R-squared",
    yaxis_range=[-1, 0.1],

)

fig.show()

In [23]:
mse_scores = {
    'XGBoost': xgb_mse,
    'Decision Tree': dt_mse,
    'Random Forest': rf_mse,
    'SVM': svm_mse
}
models = list(mse_scores.keys())
scores = list(mse_scores.values())

fig = go.Figure(data=[go.Bar(
    x=models,
    y=scores,
    text=scores,  # Add the score values to the plot
    textposition='auto',
    marker_color=['purple', 'red', 'orange', 'blue']  # Customize colors
)])

fig.update_layout(
    title="Comparison of MAE Scores for Different Regression Models (Joc)",
    xaxis_title="Models",
    yaxis_title="MSE",

)

fig.show()

In [28]:
import pandas as pd

# Assuming you have y_test and xgb_pred from your previous code

# Create a DataFrame for actual values
actual_df = pd.DataFrame({'Actual_Jsc': y_test['Jsc'].values})  # Extract values from y_test

# Create a DataFrame for predicted values, ensuring it's a 1D array
predicted_df = pd.DataFrame({'Predicted_Jsc': xgb_pred.flatten()})

# Concatenate the two DataFrames
jsc_df = pd.concat([actual_df, predicted_df], axis=1)

# Display the DataFrame
print(jsc_df.head())

   Actual_Jsc  Predicted_Jsc
0   -0.995407       0.006077
1    0.051742       0.273276
2   -1.367112       0.007453
3    0.977113      -0.040817
4   -1.577493      -0.146423


In [49]:
import plotly.graph_objects as go

# Assuming you have the 'jsc_df' DataFrame from the previous step

fig = go.Figure()

# Add trace for actual values
fig.add_trace(go.Scatter(
    x=jsc_df.index[:1000],  # Use DataFrame index for x-axis
    y=jsc_df['Actual_Jsc'],
    mode='markers',
    name='Actual',
    marker=dict(color='blue')
))

# Add trace for predicted values
fig.add_trace(go.Scatter(
    x=jsc_df.index[:1000],  # Use DataFrame index for x-axis
    y=jsc_df['Predicted_Jsc'],
    mode='markers',
    name='Predicted',
    marker=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Actual vs. Predicted Jsc Values Xgboost",
    xaxis_title="Index",
    yaxis_title="Jsc Values",
    legend_title="Data Type"
)

fig.show()

In [30]:
actual_df = pd.DataFrame({'Actual_Jsc': y_test['Jsc'].values})  # Extract values from y_test

# Create a DataFrame for predicted values, ensuring it's a 1D array
predicted_df = pd.DataFrame({'Predicted_Jsc': dt_pred.flatten()})

# Concatenate the two DataFrames
jsc_df = pd.concat([actual_df, predicted_df], axis=1)

# Display the DataFrame
print(jsc_df.head())

   Actual_Jsc  Predicted_Jsc
0   -0.995407       1.174244
1    0.051742       0.851492
2   -1.367112       0.060744
3    0.977113       1.032540
4   -1.577493       0.204673


In [46]:
import plotly.graph_objects as go

# Assuming you have the 'jsc_df' DataFrame from the previous step

fig = go.Figure()

# Add trace for actual values
fig.add_trace(go.Scatter(
    x=jsc_df.index[:1000],  # Use DataFrame index for x-axis
    y=jsc_df['Actual_Jsc'],
    mode='markers',
    name='Actual',
    marker=dict(color='red')
))

# Add trace for predicted values
fig.add_trace(go.Scatter(
    x=jsc_df.index[:1000],  # Use DataFrame index for x-axis
    y=jsc_df['Predicted_Jsc'],
    mode='markers',
    name='Predicted',
    marker=dict(color='green')
))

# Update layout
fig.update_layout(
    title="Actual vs. Predicted Jsc Values DT",
    xaxis_title="Index",
    yaxis_title="Jsc Values",
    legend_title="Data Type"
)

fig.show()

In [33]:
actual_df = pd.DataFrame({'Actual_Jsc': y_test['Jsc'].values})  # Extract values from y_test

# Create a DataFrame for predicted values, ensuring it's a 1D array
predicted_df = pd.DataFrame({'Predicted_Jsc': rf_pred.flatten()})

# Concatenate the two DataFrames
jsc_df = pd.concat([actual_df, predicted_df], axis=1)

# Display the DataFrame
print(jsc_df.head())

   Actual_Jsc  Predicted_Jsc
0   -0.995407       0.049292
1    0.051742      -0.082525
2   -1.367112      -0.231603
3    0.977113      -0.009652
4   -1.577493      -0.486415


In [41]:
import plotly.graph_objects as go

# Assuming you have the 'jsc_df' DataFrame from the previous step

fig = go.Figure()

# Add trace for actual values
fig.add_trace(go.Scatter(
    x=jsc_df.index[:1000],  # Use DataFrame index for x-axis
    y=jsc_df['Actual_Jsc'],
    mode='markers',
    name='Actual',
    marker=dict(color='black')
))

# Add trace for predicted values
fig.add_trace(go.Scatter(
    x=jsc_df.index[:1000],  # Use DataFrame index for x-axis
    y=jsc_df['Predicted_Jsc'],
    mode='markers',
    name='Predicted',
    marker=dict(color='red')
))

# Update layout
fig.update_layout(
    title="Actual vs. Predicted Jsc Values RF",
    xaxis_title="Index",
    yaxis_title="Jsc Values",
    legend_title="Data Type"
)

fig.show()

In [36]:
actual_df = pd.DataFrame({'Actual_Jsc': y_test['Jsc'].values})  # Extract values from y_test

# Create a DataFrame for predicted values, ensuring it's a 1D array
predicted_df = pd.DataFrame({'Predicted_Jsc': svm_pred.flatten()})

# Concatenate the two DataFrames
jsc_df = pd.concat([actual_df, predicted_df], axis=1)

# Display the DataFrame
print(jsc_df.head())

   Actual_Jsc  Predicted_Jsc
0   -0.995407      -0.100899
1    0.051742       0.040242
2   -1.367112      -0.002491
3    0.977113       0.060680
4   -1.577493      -0.273983


In [40]:
import plotly.graph_objects as go

# Assuming you have the 'jsc_df' DataFrame from the previous step

fig = go.Figure()

# Add trace for actual values
fig.add_trace(go.Scatter(
    x=jsc_df.index[:1000],  # Use DataFrame index for x-axis
    y=jsc_df['Actual_Jsc'],
    mode='markers',
    name='Actual',
    marker=dict(color='blue')
))

# Add trace for predicted values
fig.add_trace(go.Scatter(
    x=jsc_df.index[:1000],  # Use DataFrame index for x-axis
    y=jsc_df['Predicted_Jsc'],
    mode='markers',
    name='Predicted',
    marker=dict(color='purple')
))

# Update layout
fig.update_layout(
    title="Actual vs. Predicted Jsc Values SVM",
    xaxis_title="Index",
    yaxis_title="Jsc Values",
    legend_title="Data Type"
)

fig.show()